# VizTool
In this notebook, we show the features of the `VizTool`, an interactive tool to visualize events and main parameters.

In [1]:
import cait as ai

## Mock data and DataHandler object
The HDF5 file used in this notebook can be created using the following commands. If you already have an HDF5 file and a corresponding `DataHandler` object, you can skip this step.

In [ ]:
test_data = ai.data.TestData(filepath='./mock_001', duration=10000)
test_data.generate()
dh = ai.DataHandler(channels=[0,1])
dh.convert_dataset(path_rdt='./', fname='mock_001', path_h5='./')
dh.set_filepath(path_h5='./', fname='mock_001')

dh.calc_mp()
dh.calc_mp(type='testpulses')
dh.calc_mp(type='noise')

dh.calc_additional_mp(type='events', no_of=True)
dh.calc_additional_mp(type='testpulses', no_of=True)
dh.calc_additional_mp(type='noise', no_of=True)

In [ ]:
# create DataHandler instance
dh = ai.DataHandler(channels=[0,1])
dh.set_filepath(path_h5='./', fname='mock_001', appendix=True)

## Visualization
We now have to decide, which data to visualize. This is handled with a `datasets` dictionary: The keys of the dictionary are the labels that will be displayed on the plot. The value is always a list of the name of the dataset, the index in the first dimension (channel index) and last dimension (feature index). If there is no channel or feature index to be set, the value in the list can be put to `None`. This might look overwhelming at first but most of the time, you will only use exactly this dictionary.

In [10]:
datasets = {
    'Time (h)': ['hours', None, None],
    'Pulse Height Phonon (V)': ['pulse_height', 0, None],
    'Pulse Height Light (V)': ['pulse_height', 1, None],
    'Rise Time Phonon (ms)': ['rise_time', 0, None],
    'Rise Time Light (ms)': ['rise_time', 1, None],
    'Decay Time Phonon (ms)': ['decay_time', 0, None],
    'Decay Time Light (ms)': ['decay_time', 1, None],
    'Onset Phonon (ms)': ['onset', 0, None],
    'Onset Light (ms)': ['onset', 1, None],
    'Slope Phonon (V)': ['slope', 0, None],
    'Slope Light (V)': ['slope', 1, None],
    'Variance Phonon (V^2)': ['var', 1, None],
    'Variance Light (V^2)': ['var', 0, None],
    'Mean Phonon (V)': ['mean', 0, None],
    'Mean Light (V)': ['mean', 1, None],
    'Skewness Phonon': ['skewness', 0, None],
    'Skewness Light': ['skewness', 1, None],
}

Using this dictionary, we are ready to construct the `vizTool` object. Here, we use the already existing `DataHandler` as data source. Note, however, that you could also specify the path to an `*.h5` or `*.csv` file.

In [19]:
viz = ai.VizTool(datahandler=dh, datasets=datasets, group="events")

You might not want to plot *all* the data because you might have already performed some cuts, for example. We can choose to show only certain events from the file by handing the indices of these events to the instance of the `VizTool`. Let us define some arbitrary cut to illustrate this procedure:

In [20]:
some_cut = ai.cuts.LogicalCut(dh.get('events', 'pulse_height')[0] < 5)  # cut on the phonon channel

viz.set_idx(some_cut.get_idx())

We are now ready to display the tool. The `show` method takes one argument `template` which you can use to style the tool to your likings.

In [ ]:
viz.show(template='ggplot2')

![](media/vizTool_upper.png)
![](media/vizTool_lower.png)

The `VizTool`'s main component is a scatter plot surrounded by histograms for the x- and y-values that are scatter plotted. You can choose the axis using the x- and y-dropdown-menus. The `color`-dropdown lets you choose a coloring for the scatter plot, effectively giving you a third plot axis. You can choose to have a heatmap instead of a scatter plot by using the radio-buttons on top. The scale (log or linear) for the heatmap and histograms can be selected using the respective radio-button.

In the scatter plot, we can select and plot events (which show up in the bottom part of the tool) and cut them away (using box or lasso select and the button `Cut Selected`). Note that they will not be deleted from the HDF5 file but only from the current `vizTool` instance.

In the textbox, you can define a name for the selected events (you need to press `Enter`) and then store the cut flag (a boolean array) for the selected event in the HDF5 file (`Save Selected`) or export the index list as CSV (`Export Selected`). 

You can calculate a preview of a standard event of your selected events with the `Calc SEV` button. 
Once you choose a list of events for which you want to calculate a proper standard event, you can access the saved list from the HDF5 file with the `DataHandler.get` method. You can define the index list with a `LogicalCut` object and then use it in the `calc_sev` method:

In [ ]:
sev_cuts = ai.cuts.LogicalCut(dh.get('events', 'sev_cuts')[0])

dh.calc_sev(use_idx=sev_cuts.get_idx())